In [1]:
%load_ext autoreload
%autoreload 2
from scripts.agents import Tool, InternalTool, CustomAgent, CustomGraph, InternalToolInput
from dotenv import load_dotenv
load_dotenv()

c:\Users\cgwed\anaconda3\envs\llm_hub\Lib\site-packages\langgraph\graph\graph.py:30: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langgraph.pregel import Channel, Pregel
c:\Users\cgwed\anaconda3\envs\llm_hub\Lib\site-packages\pydantic\_internal\_generate_schema.py:547: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given 

True

In [3]:
def test_internal_tool():
    # Define a simple function to be used as the tool's function
    def example_function(query: str, additional_arg: str = "") -> str:
        return f"Processed query: {query}, Additional arg: {additional_arg}"

    # Create an instance of InternalTool
    tool = InternalTool(
        name="ExampleTool",
        description="An example tool for testing",
        function=example_function
    )


    # Test the prepare method
    prepared_tool = tool.prepare(additional_arg="test_arg")

    # Test the _run method
    result1 = tool._run(query="Hello, world!")

    # Test the _run method with overriding prepared args
    result2 = tool._run(query="Hello, again!", additional_arg="override_arg")
    return prepared_tool, result1, result2

In [4]:
tool, result1, result2 = test_internal_tool()

AttributeError: 'InternalTool' object has no attribute 'prepare'

In [9]:
from unittest.mock import Mock
mock_llm = Mock()

# Create a mock tool
def mock_function(arg1, arg2):
    return f"Result: {arg1}, {arg2}"
present_args = {'preset_arg1': 'preset_arg1'}
run_arg_description = {'arg1': 'arg1 descriptionn', 'arg2': 'arg2 description'}
mock_tool = InternalTool(name="MockTool", description='Mock tool for testing', function = mock_function, run_arg_descriptions=run_arg_description, present_args=present_args)


In [10]:
from langchain_community.agent_toolkits.load_tools import load_tools
tools = [mock_tool]
kwargs = {'custom_arg': 'test_value'}

import inspect

def _prepare_tools(tools, llm):
    """
    If a tool has a "prepare" method, 
    set its arguments with the arguments from kwargs that match

    Args:
        tools (_type_): _description_
        llm (_type_): _description_

    Returns:
        _type_: _description_
    """
    prepared_tools = []
    for tool in tools:
        if callable(getattr(tool, 'prepare', None)):
            # Get the parameters of the prepare method
            prepare_params = inspect.signature(tool.prepare).parameters
            # Filter kwargs to only include those that mathc prepare's parameters
            filtered_kwargs = {k: v for k, v in kwargs.items() if k in prepare_params}
            prepared_tool = tool.prepare(**filtered_kwargs)
        else:
            prepared_tool = tool
        prepared_tools.append(prepared_tool)
    return load_tools([], llm=llm) + prepared_tools



In [11]:
prepared_tools = _prepare_tools(tools=tools, llm=mock_llm)

In [13]:
from langchain_mistralai import ChatMistralAI
from langchain_aws import ChatBedrock
# Define the Bedrock client using LangChain
#llm = ChatMistralAI(
#    model="mistral-small-latest"
#  )  # Replace with the specific Bedrock model you want to use

llm = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0",
    region_name='us-east-1',
    model_kwargs = {
        'temperature': 0
    }
)


In [14]:
from langchain.agents import AgentType
from langgraph.graph import START, END

# Define tools for each operation
def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b


add_tool = InternalTool(
        name="Addition",
        description="Add two numbers together",
        function=add
    )

subtract_tool = InternalTool(
        name="SubtractTool",
        description="Subtract two numbers",
        function=subtract
    )

multiply_tool = InternalTool(
        name="ExampleTool",
        description="An example tool for testing",
        function=multiply
    )

# Create CustomAgent instances
add_agent = CustomAgent("AddAgent", llm, [add_tool], AgentType.ZERO_SHOT_REACT_DESCRIPTION)
subtract_agent = CustomAgent("SubtractAgent", llm, [subtract_tool], AgentType.ZERO_SHOT_REACT_DESCRIPTION)
multiply_agent = CustomAgent("MultiplyAgent", llm, [multiply_tool], AgentType.ZERO_SHOT_REACT_DESCRIPTION)



In [15]:
def make_available_nodes(nodes):
    available_nodes = {node.name: node for node in nodes}
    available_nodes['START'] = START
    available_nodes['END'] = END
    return available_nodes
available_nodes = make_available_nodes([add_agent, subtract_agent, multiply_agent])

In [16]:
outline = {'START': ['AddAgent', 'SubtractAgent'],
           'AddAgent': ['MultiplyAgent', 'END'],
           'SubtractAgent': ['MultiplyAgent', 'END'],
           'MultiplyAgent': ['END']}

In [17]:
available_nodes

{'AddAgent': <scripts.agents.CustomAgent at 0x22d57962c30>,
 'SubtractAgent': <scripts.agents.CustomAgent at 0x22d55d288f0>,
 'MultiplyAgent': <scripts.agents.CustomAgent at 0x22d55d283e0>,
 'START': '__start__',
 'END': '__end__'}

In [1]:
graph = CustomGraph(outline=outline)
graph.create_graph(available_nodes=available_nodes)

NameError: name 'CustomGraph' is not defined

In [5]:
input = ("Add 3 and 4, then multiply the result by 2")
graph.graph({"input": input})

NameError: name 'graph' is not defined

In [6]:
graph.graph

NameError: name 'graph' is not defined

In [7]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
events = graph.graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Fetch the UK's GDP over the past 5 years,"
                " then draw a line graph of it."
                " Once you code it up, finish."
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150},
)

NameError: name 'graph' is not defined

In [8]:

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
# Run a sample query
#response = llm.invoke(messages)
#print(response)


In [9]:
from langchain.agents import AgentType


agent = CustomAgent(
        name="TestAgent",
        llm=llm,
        tools=[mock_tool],
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=False,
        custom_arg="test_value"
    )

NameError: name 'llm' is not defined

In [10]:
agent.tools

NameError: name 'agent' is not defined

In [11]:
NodeState = {
    "messages": [
        {"role": "user", "content": "Hello, I need help with my project."},
        {"role": "assistant", "content": "Sure, I'd be happy to help. What's your project about?"}
    ],
    "current_agent": "project_manager",
    "available_nodes": ["developer", "designer", "qa_tester"],
    "node_types": {
        "project_manager": "management",
        "developer": "technical",
        "designer": "creative",
        "qa_tester": "quality_assurance"
    },
    "finished": False
}
agent.invoke(query=NodeState)

NameError: name 'agent' is not defined